Import the libraries

In [30]:
from dotenv import load_dotenv
from langchain import hub
from langchain.agents import (
    AgentExecutor,
    create_react_agent,
)
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
import os
import datetime

load_dotenv()

True

Define tools

In [49]:
from test.test_typing import Annotated


@tool("get_current_datetime")
def get_current_datetime() -> str :
    """Get current date and time"""
    return datetime.datetime.now().strftime('%m/%d/%Y, %H:%M:%S')

@tool("convert_to_japanse_datetime_custom")
def convert_to_japanse_datetime(a: Annotated[str, 'date string in ISO format']) -> str :
    """Convert date string to custom japanese date time"""
    dt = datetime.datetime.fromisoformat(a)
    return f"{dt.year}年{dt.month}月{dt.day}日　{dt.hour}時{dt.minute}分{dt.second}秒 hehe"

Initialized OpenAI LLM

In [ ]:
# Initialize a ChatOpenAI model
llm = ChatOpenAI(
    model="gpt-4o-mini",
    api_key=os.getenv('OPENAI_KEY'),
)

tools = [
    get_current_datetime,
    convert_to_japanse_datetime
]

prompt = hub.pull("hwchase17/react",api_key=os.getenv('LANGSMITH_KEY'),)
agent = create_react_agent(
    llm=llm,
    tools=tools,
    prompt=prompt,
    stop_sequence=True,
)


# Create an agent executor from the agent and tools
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent,
    tools=tools,
    verbose=True,
)

Testing

In [51]:
response = agent_executor.invoke(input={"input": ["Tell me the current datetime and in japanese format"]})
print("response:", response)



> Entering new AgentExecutor chain...
I need to get the current datetime first, and then convert it to Japanese format.  
Action: get_current_datetime  
Action Input: None  02/21/2025, 20:55:28I now have the current datetime in standard format, which is "02/21/2025, 20:55:28".  
Now I need to convert this datetime into Japanese format.  
Action: convert_to_japanse_datetime_custom  
Action Input: "2025-02-21T20:55:28"  2025年2月21日　20時55分28秒 heheI now know the final answer, which includes both the current datetime and its Japanese format.  
Final Answer: The current datetime is 02/21/2025, 20:55:28 and in Japanese format, it is 2025年2月21日　20時55分28秒.

> Finished chain.
response: {'input': ['Tell me the current datetime and in japanese format'], 'output': 'The current datetime is 02/21/2025, 20:55:28 and in Japanese format, it is 2025年2月21日\u300020時55分28秒.'}
